In [1]:
# pip install geopandas matplotlib pillow

   ANN_CD ANN_NM        F_AREA  \
0     140    활엽수  6.842632e+03   
1     140    활엽수  7.659524e+03   
2     140    활엽수  1.479724e+04   
3     140    활엽수  7.381741e+06   
4     140    활엽수  7.381741e+06   

                                            geometry  
0  POLYGON ((216245.458 615321.718, 216225.433 61...  
1  POLYGON ((218077.280 616906.793, 218063.025 61...  
2  POLYGON ((218167.514 616854.159, 218167.514 61...  
3  POLYGON ((218167.514 614857.809, 218167.514 61...  
4  POLYGON ((217775.435 616835.831, 217775.435 61...  


In [11]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image
import io
from tqdm import tqdm

def process_json_file(json_file, output_folder):
    # JSON 파일 불러오기
    gdf = gpd.read_file(json_file)

    # 이미지 크기 설정
    image_size = (512, 512)

    # 폴리곤을 하나의 이미지로 합치기
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.patch.set_facecolor('white')  # 배경색을 흰색으로 설정

    for idx, row in gdf.iterrows():
        # 각 폴리곤을 'ANN_CD' 값에 따라 다른 그레이스케일 톤으로 채움
        gray_value = row['ANN_CD']/gdf['ANN_CD'].max()
        gdf[gdf.index == idx].plot(ax=ax, color=(gray_value, gray_value, gray_value))

    # 축 제거 및 이미지 크기 조정
    plt.axis('off')
    plt.xlim(gdf.total_bounds[0], gdf.total_bounds[2])
    plt.ylim(gdf.total_bounds[1], gdf.total_bounds[3])
    ax.set_aspect('equal', adjustable='box')

    # 이미지로 변환
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')
    buf.seek(0)
    img = Image.open(buf)
    img = img.resize(image_size, Image.Resampling.LANCZOS)
    buf.close()
    plt.close(fig)

    # 이미지 파일 이름 설정 (JSON 파일 이름 기반)
    image_file_name = os.path.splitext(os.path.basename(json_file))[0] + '.png'
    image_path = os.path.join(output_folder, image_file_name)

    # 이미지 저장
    img.save(image_path)

# 폴더 경로 설정
folder_path = [#'Dataset\Training\label\AP10_City_Json','Dataset\Training\label\AP10_Forest_Json','Dataset\Training\label\SN10_Forest_Json'
               'Dataset\Training\label\AP25_City_Json','Dataset\Training\label\AP25_Forest_Json',
               'Dataset\Validation\label\AP10_City_Json','Dataset\Validation\label\AP10_Forest_Json','Dataset\Validation\label\AP25_City_Json','Dataset\Validation\label\AP25_Forest_Json','Dataset\Validation\label\SN10_Forest_Json']
for path in folder_path:
    output_folder = path.replace('Json', 'GT')
    # output_folder가 존재하지 않으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)
    # 폴더 내의 모든 JSON 파일에 대해 작업 수행
    for file in tqdm(os.listdir(path)):
        if file.endswith('.json'):
            json_file_path = os.path.join(path, file)
            process_json_file(json_file_path, output_folder)


 87%|████████▋ | 11083/12800 [34:15<02:33, 11.19it/s]  